In [11]:
import tensorflow as tf
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [4]:
x_data = np.concatenate((x_train, x_test), axis=0)
y_data = np.concatenate((y_train, y_test), axis=0)


num_samples = x_data.shape[0]



train_size = int(0.70 * num_samples)  # 70% for training
val_size = int(0.15 * num_samples)  # 15% for validation
test_size = num_samples - train_size - val_size  # 15% for testing

x_train = x_data[:train_size]
y_train = y_data[:train_size]
x_val = x_data[train_size:train_size+val_size]
y_val = y_data[train_size:train_size+val_size]
x_test = x_data[train_size+val_size:]
y_test = y_data[train_size+val_size:]

In [5]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.map(preprocess).batch(32).prefetch(tf.data.experimental.AUTOTUNE)


print(f"Train Data: {len(x_train)}")
print(f"Validation Data: {len(x_val)}")
print(f"Test Data: {len(x_test)}")

Train Data: 42000
Validation Data: 9000
Test Data: 9000


In [18]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import BatchNormalization
from keras import regularizers




model = Sequential()

model.add(Conv2D(32, (3, 3), padding="same", activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.50))

model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.50))


model.add(Flatten())

model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(512, activation='relu', kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(512, activation='relu',
                kernel_regularizer=regularizers.l1_l2(l1=0.005, l2=0.005)))

model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 batch_normalization (Batch  (None, 30, 30, 32)        128       
 Normalization)                                                  
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 15, 15, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 15, 15, 64)       

In [19]:
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_val, y_val), shuffle=True)

#Saving the neural network
model_structure = model.to_json()
with open("model_structure.json", "w") as json_file:
    json_file.write(model_structure)

#Save the trained weights
model.save_weights("model_weights.h5")

Epoch 1/30
1563/1563 [==============================] - 291s 184ms/step - loss: 5.9854 - accuracy: 0.2456 - val_loss: 2.1622 - val_accuracy: 0.3354
Epoch 2/30
1563/1563 [==============================] - 284s 182ms/step - loss: 2.1891 - accuracy: 0.3339 - val_loss: 2.0511 - val_accuracy: 0.3719
Epoch 3/30
1563/1563 [==============================] - 285s 182ms/step - loss: 2.1077 - accuracy: 0.3783 - val_loss: 2.1683 - val_accuracy: 0.3737
Epoch 4/30
1563/1563 [==============================] - 283s 181ms/step - loss: 2.0418 - accuracy: 0.4324 - val_loss: 2.0424 - val_accuracy: 0.4462
Epoch 5/30
1563/1563 [==============================] - 284s 182ms/step - loss: 1.9757 - accuracy: 0.4751 - val_loss: 1.8106 - val_accuracy: 0.5416
Epoch 6/30
1563/1563 [==============================] - 284s 182ms/step - loss: 1.9337 - accuracy: 0.4989 - val_loss: 1.8246 - val_accuracy: 0.5381
Epoch 7/30
1563/1563 [==============================] - 284s 182ms/step - loss: 1.8988 - accuracy: 0.5149 - val_

In [20]:
from keras.models import model_from_json
from keras.preprocessing import image

class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

with open('model_structure.json', 'r') as file:
    model_structure = file.read()

model= model_from_json(model_structure)

model.load_weights("model_weights.h5")

In [21]:
img = image.load_img('deer.jpg', target_size=(32, 32))
image_test = image.img_to_array(img) /255
list_images = np.expand_dims(image_test, axis=0)

results = model.predict(list_images)

single_result = results[0]

most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

class_labels[most_likely_class_index]

print("This is a {}->{}".format(class_labels[most_likely_class_index], class_likelihood))

1/1 [==============================] - 0s 147ms/step
This is a frog->0.4289560914039612


In [22]:
img = image.load_img('cat.jpg', target_size=(32, 32))
image_test = image.img_to_array(img) /255
list_images = np.expand_dims(image_test, axis=0)

results = model.predict(list_images)

single_result = results[0]

most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result[most_likely_class_index]

class_labels[most_likely_class_index]

print("This is a {}->{}".format(class_labels[most_likely_class_index], class_likelihood))

1/1 [==============================] - 0s 23ms/step
This is a frog->0.42381006479263306
